In [22]:
# Import required libraries
import pandas as pd
import numpy as np
from pathlib import Path
import logging
from datetime import datetime
from tqdm import tqdm
import pytz

# Configure logging
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
quality_report = f'../reports/sector_quality_{timestamp}.log'

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(),
        logging.FileHandler(quality_report)
    ]
)

# Configuration
START_DATE = '2015-01-01'
END_DATE = datetime.now().strftime('%Y-%m-%d')
UTC = pytz.UTC

# Required columns for validation
REQUIRED_COLUMNS = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']

# Define GICS sectors and their ETFs
SECTOR_ETF_MAP = {
    'XLK': 'Information Technology',    # Technology sector benchmark
    'XLF': 'Financials',               # Banks, Insurance, Investment firms
    'XLV': 'Healthcare',               # Healthcare providers, Biotech
    'XLE': 'Energy',                   # Oil & Gas companies
    'XLY': 'Consumer Discretionary',   # Retail, Automotive, Media
    'XLP': 'Consumer Staples',         # Food & Beverage
    'XLI': 'Industrials',              # Aerospace & Defense
    'XLB': 'Materials',                # Chemicals, Mining
    'XLU': 'Utilities',                # Electric & Gas utilities
    'XLRE': 'Real Estate',             # REITs & Property management
    'XLC': 'Communication Services'     # Telecom services & Media
}

# Validation configuration
VALIDATION_CONFIG = {
    'min_trading_days': 1000,  # Minimum number of trading days required
    'max_missing_pct': 5,      # Maximum percentage of missing data allowed
    'outlier_threshold': 5     # Standard deviations for outlier detection
}


In [23]:
# Utility functions for data validation
def validate_data_quality(df, name):
    """Validate data quality metrics."""
    try:
        # Check date range
        if len(df) < VALIDATION_CONFIG['min_trading_days']:
            raise ValueError(f"Insufficient trading days ({len(df)})")
        
        # Check missing data
        missing_pct = (df[REQUIRED_COLUMNS].isnull().sum() / len(df) * 100).max()
        if missing_pct > VALIDATION_CONFIG['max_missing_pct']:
            raise ValueError(f"High missing data ({missing_pct:.1f}%)")
        
        # Check for invalid values
        for col in ['Open', 'High', 'Low', 'Close']:
            if (df[col] <= 0).any():
                raise ValueError(f"Invalid {col} values (<=0)")
            if df[col].isnull().any():
                raise ValueError(f"Missing {col} values")
        
        if (df['Volume'] < 0).any():
            raise ValueError(f"Invalid Volume values (<0)")
        
        # Check price relationships with tolerance for equal values
        invalid_price = (
            (df['High'] < df['Low']) |  # High should never be less than Low
            (df['High'] < df['Open']) |  # High should never be less than Open
            (df['High'] < df['Close']) |  # High should never be less than Close
            (df['Low'] > df['Open']) |   # Low should never be greater than Open
            (df['Low'] > df['Close'])     # Low should never be greater than Close
        )
        
        # Remove cases where values are equal (within floating point precision)
        equal_cases = (
            (abs(df['High'] - df['Low']) < 1e-10) |
            (abs(df['High'] - df['Open']) < 1e-10) |
            (abs(df['High'] - df['Close']) < 1e-10) |
            (abs(df['Low'] - df['Open']) < 1e-10) |
            (abs(df['Low'] - df['Close']) < 1e-10)
        )
        invalid_price = invalid_price & ~equal_cases
        
        if invalid_price.any():
            invalid_rows = df[invalid_price]
            print(f"\nInvalid price relationships in {name}:")
            print(invalid_rows[['Date', 'Open', 'High', 'Low', 'Close']].head())
            raise ValueError(f"Invalid price relationships ({len(invalid_rows)} rows)")
        
        # Check for outliers
        for col in ['Open', 'High', 'Low', 'Close']:
            mean = df[col].mean()
            std = df[col].std()
            threshold = VALIDATION_CONFIG['outlier_threshold'] * std
            outliers = df[abs(df[col] - mean) > threshold]
            if len(outliers) > len(df) * 0.05:  # More than 5% outliers
                raise ValueError(f"High number of outliers in {col}")
        
        return True
        
    except Exception as e:
        raise ValueError(f"Data quality validation failed: {str(e)}")

def process_sector_data(df, sector_name, etf):
    """Process sector ETF data."""
    try:
        # Convert date to datetime with UTC
        df['Date'] = pd.to_datetime(df['Date'], utc=True)
        
        # Create a new DataFrame with only required columns to avoid SettingWithCopyWarning
        processed_df = pd.DataFrame()
        for col in REQUIRED_COLUMNS:
            processed_df[col] = df[col]
        
        # Add sector information
        processed_df['Sector'] = sector_name
        processed_df['Ticker'] = etf
        
        # Sort by date
        processed_df = processed_df.sort_values('Date')
        
        # Validate data quality
        validate_data_quality(processed_df, f"{sector_name} ({etf})")
        
        return processed_df
        
    except Exception as e:
        raise ValueError(f"Error processing {sector_name} ({etf}): {str(e)}")

def validate_columns(df, required_cols, name):
    """Validate that all required columns are present."""
    missing_cols = set(required_cols) - set(df.columns)
    if missing_cols:
        raise ValueError(f"{name}: Missing columns: {missing_cols}")
    return True


In [24]:
# Main execution
print("Starting sector data processing...\n")

# Dictionary to store processed data
sector_data = {}
validation_results = {'passed': [], 'failed': []}

# Process each sector ETF
for etf, sector in tqdm(SECTOR_ETF_MAP.items(), desc="Processing sectors"):
    try:
        logging.info(f"Processing {etf} ({sector})")
        
        # Load data
        file_path = Path(f'../data/sectors/{etf}.csv')
        if not file_path.exists():
            raise FileNotFoundError(f"No data file found for ETF {etf}")
        
        df = pd.read_csv(file_path)
        
        # Validate columns
        validate_columns(df, REQUIRED_COLUMNS, f"{sector} ({etf})")
        
        # Process data
        df = process_sector_data(df, sector, etf)
        
        # Store processed data
        sector_data[sector] = df
        validation_results['passed'].append(sector)
        
        # Print summary
        print(f"\n{etf} ({sector}) Summary:", flush=True)
        print(f"- Date Range: {df['Date'].min():%Y-%m-%d} to {df['Date'].max():%Y-%m-%d}")
        print(f"- Trading Days: {len(df)}")
        print(f"- File: {file_path}")
        
    except Exception as e:
        logging.error(f"Error processing {etf} ({sector}): {str(e)}")
        validation_results['failed'].append(sector)
        continue

# Print sector coverage
print("\nSector Coverage:", flush=True)
print(f"- Total Sectors: {len(SECTOR_ETF_MAP)}")
print(f"- Loaded Sectors: {len(validation_results['passed'])}")
if validation_results['failed']:
    print(f"- Failed Sectors: {validation_results['failed']}")

# Create merged dataset if we have data
if sector_data:
    try:
        print("\nCreating merged dataset...")
        merged_data = []
        
        for sector, df in sector_data.items():
            merged_data.append(df)
        
        merged_df = pd.concat(merged_data, ignore_index=True)
        merged_df = merged_df.sort_values(['Date', 'Sector'])
        
        # Save merged data
        output_path = Path('../data/processed/sector_indices_merged.csv')
        output_path.parent.mkdir(parents=True, exist_ok=True)
        merged_df.to_csv(output_path, index=False)
        logging.info(f"Saved merged sector data to {output_path}")
        
        # Print summary
        print("\nSector Data Summary:")
        print(f"- Total sectors: {len(sector_data)}")
        if len(merged_df) > 0:
            print(f"- Date range: {merged_df['Date'].min():%Y-%m-%d} to {merged_df['Date'].max():%Y-%m-%d}")
            print(f"- Trading days: {len(merged_df['Date'].unique())}")
            print("\nRows per sector:")
            sector_counts = merged_df.groupby('Sector').size()
            for sector, count in sector_counts.items():
                print(f"- {sector}: {count:,} rows")
        else:
            print("No data to summarize")
        
    except Exception as e:
        logging.error(f"Error creating merged dataset: {str(e)}")

print("\nProcessing complete!")


Starting sector data processing...



Processing sectors:   0%|          | 0/11 [00:00<?, ?it/s]2025-07-26 07:14:55,711 - INFO - Processing XLK (Information Technology)



XLK (Information Technology) Summary:


2025-07-26 07:14:55,773 - INFO - Processing XLF (Financials)


- Date Range: 2015-01-02 to 2025-07-25
- Trading Days: 2656
- File: ..\data\sectors\XLK.csv

XLF (Financials) Summary:


...

- Date Range: 2015-10-08 to 2025-07-25
- Trading Days: 2463
- File: ..\data\sectors\XLRE.csv

XLC (Communication Services) Summary:


Processing sectors: 100%|██████████| 11/11 [00:00<00:00, 16.92it/s]

- Date Range: 2018-06-19 to 2025-07-25
- Trading Days: 1785
- File: ..\data\sectors\XLC.csv

Sector Coverage:
- Total Sectors: 11
- Loaded Sectors: 11

Creating merged dataset...



2025-07-26 07:14:56,651 - INFO - Saved merged sector data to ..\data\processed\sector_indices_merged.csv



Sector Data Summary:
- Total sectors: 11
- Date range: 2015-01-02 to 2025-07-25
- Trading days: 2656

Rows per sector:
- Communication Services: 1,785 rows
- Consumer Discretionary: 2,656 rows
- Consumer Staples: 2,656 rows
- Energy: 2,656 rows
- Financials: 2,656 rows
- Healthcare: 2,656 rows
- Industrials: 2,656 rows
- Information Technology: 2,656 rows
- Materials: 2,656 rows
- Real Estate: 2,463 rows
- Utilities: 2,656 rows

Processing complete!
